In [ ]:
import sys
from PyQt5.QtGui import QImage
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QFileDialog, QLineEdit, QPlainTextEdit
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
from tensorflow.keras.models import load_model
import numpy as np
import cv2


class ImageClassifier(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(100, 100, 1200, 800)
        self.setWindowTitle('LeukoAI')
        self.setWindowFlag(Qt.FramelessWindowHint)
        self.setStyleSheet("background-color: #F7DC6F;")

        self.image_label = QLabel(self)
        self.image_label.setGeometry(50, 50, 500, 500)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("border: 10px solid #000000; background-color: #ffffff; border-radius: 5")
                
        self.classification_label = QLabel(self)
        self.classification_label.setStyleSheet("font-weight: bold")
        font = self.classification_label.font()
        font.setPointSize(14)
        self.classification_label.setFont(font)
        
        self.a = QPlainTextEdit(self)
        font1 = self.a.font()
        font1.setPointSize(12)
        self.a.setFont(font1)
        self.a.setStyleSheet("border: none; font-weight: bold")

        self.b = QPlainTextEdit(self)
        self.b.setFont(font1)
        self.b.setStyleSheet("border: none; font-weight: bold")
        
        self.c = QPlainTextEdit(self)
        self.c.setFont(font1)
        self.c.setStyleSheet("border: none; font-weight: bold")
        
        self.d = QPlainTextEdit(self)
        self.d.setFont(font1)
        self.d.setStyleSheet("border: none; font-weight: bold")
        
        self.classification_button = QPushButton('Classify', self)
        self.classification_button.setGeometry(310, 600, 200, 50)
        self.classification_button.clicked.connect(self.classifyImage)
        self.classification_button.setStyleSheet("background-color: #2ECC71; color: #ffffff; border-radius: 20")

        self.exit_button = QPushButton('Exit', self)
        self.exit_button.setGeometry(195, 670, 200, 50)
        self.exit_button.clicked.connect(self.exitApp)
        self.exit_button.setStyleSheet("background-color: #FF0000; color: #ffffff; border-radius: 20")
        
        self.open_button = QPushButton('Open', self)
        self.open_button.setGeometry(70, 600, 200, 50)
        self.open_button.clicked.connect(self.openImage)
        self.open_button.setStyleSheet("background-color: #3498DB; color: #ffffff; border-radius: 20")

        self.statusBar().showMessage('LeukoAI')

        self.model = load_model('/home/prostoi/Загрузки/converted_keras/nauryz.h5')
        
    def exitApp(self):
        window.close()
    
    def openImage(self):
        options = QFileDialog.Options()
        options |= QFileDialog.DontUseNativeDialog
        filename, _ = QFileDialog.getOpenFileName(self, 'Open Image', '',
                                                  'Images (*.png *.xpm *.jpg *.bmp);;All Files (*)', options=options)
        if filename:
            self.image = cv2.imread(filename)
            self.image = cv2.resize(self.image, (500, 500))
            self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
            self.displayImage()

    def displayImage(self):
        height, width, channel = self.image.shape
        bytesPerLine = 3 * width
        qimage = QImage(self.image.data, width, height, bytesPerLine, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        self.image_label.setPixmap(pixmap)
        self.statusBar().showMessage('Изображение открыто')

    def classifyImage(self):
        try:
            image = cv2.resize(self.image, (224, 224))
            image = np.array(image) / 255.0
            image = image.reshape(-1, 224, 224, 3)
            prediction = self.model.predict(image)
            spisok = [prediction[0][0], prediction[0][1], prediction[0][2], prediction[0][3]]
            if max(spisok) == prediction[0][0]:
                self.classification_label.setText('Normal')
                self.classification_label.move(850, 60)
                self.classification_label.setStyleSheet('color: green;')
                self.a.insertPlainText("""К счастью, у вас нет никаких проблем с глазами. Пожалуйста, не забывайте проводить диагностику такого рода раз в 3 месяца. \n\n Правила сохранения зрения:
Хорошо высыпаться. Регулярный недосып плохо влияет на состояние зрительного нерва, вследствие чего качество зрения заметно снижается. \n
Давать глазам отдых в течение активного дня. Во время чтения, просмотра телевизора, работы за компьютером необходимо делать паузы (10-15 минут). Желательно одну-две таких паузы посвящать специальным упражнениям для глаз. \n
Смотреть телевизор и читать книги важно в хорошо освещённом помещении. Привычка смотреть телевизор в темноте губительна для здоровья глаз. \n
Читать в правильном положении. Категорически нельзя читать книги во время движения (в транспорте, при ходьбе). Читать надо сидя, не стоит привыкать к чтению лёжа. \n
Не щуриться. Эта привычка не только портит внешний вид, но и отрицательно влияет на глаза. \n
Употреблять в пищу продукты, богатые витаминами А, Е, С. \n
Чаще отдыхать, гулять на свежем воздухе. \n
Регулярно посещать офтальмолога для профилактических осмотров.""")
                self.a.resize(600, 700)
                self.a.move(600, 110)
            if max(spisok) == prediction[0][2]:
                self.classification_label.setText('Glaucoma')
                self.classification_label.move(850, 60)
                self.classification_label.setStyleSheet('color: red;')
                self.b.insertPlainText("""К сожалению, у вас обнаружена глаукома. Глаукома – одно из наиболее тяжелых глазных заболеваний, характеризующееся быстрым прогрессированием и приводящее к необратимой слепоте. \n
Основные причины заболевания \n
Внутриглазное давление является основным аспектом оценки здоровья пациентов, которые находятся в группе риска глаукомы. В норме давление в глазу составляет 18-24 мм рт. ст. за счет баланса формирования внутриглазной жидкости и ее оттока. При глаукоме данный процесс нарушается, в результате образовывающаяся жидкость накапливается, вследствие чего ВГД повышается.

Повышенное давление негативно влияет на зрительный нерв, ухудшается его кровоснабжение, спустя некоторое время нерв отмирает.

Вначале патологические изменения отражаются на боковом зрении, которое начинает резко снижаться. Со временем ухудшается острота зрительной функции и развивается необратимая слепота.

Только раннее диагностирование заболевания позволяет провести эффективное лечение и предотвратить развитие слепоты.   \n
Лечение
Полностью избавиться от глаукомы на сегодняшний день не представляется возможным, но при корректном и своевременном лечении удается замедлить прогрессирование болезни. Выбор метода терапии напрямую зависит от тяжести патологических изменений, присутствующих симптомов, формы глаукомы и заключается в применении медикаментозных средств, хирургического вмешательства или лазерной терапии.

Основной задачей проводимого лечения является снижение внутриглазного давления, нормализация кровоснабжения зрительного нерва, улучшение метаболических процессов в тканевых структурах глазного яблока.
                """)
                self.b.resize(600, 700)
                self.b.move(600, 110)
            if max(spisok) == prediction[0][3]:
                self.classification_label.setText('Retino')
                self.classification_label.setStyleSheet('color: red;')                
                self.classification_label.move(850, 60)
                self.c.insertPlainText("""К сожалению, у вас обнаружена ретинобластома. Ретинобластома — это онкологическое заболевание глаза. Ретинобластома формируется в сетчатке глаза. Сетчатка — тонкий слой нервной ткани на задней поверхности глаза. Клетки сетчатки распознают свет и цвет. Ретинобластома может поражать один глаз (односторонняя, наиболее распространенная) или оба глаза (двусторонняя). \n
Причины ретинобластомы
В 50-60% случаев ретинобластомы имеют генетическое (врожденное) происхождение. Более половины детей с ретинобластомой рождается от родителей, имевших в детстве аналогичное заболевание. Врожденная ретинобластома обычно выявляется у детей в возрасте до 30 месяцев. В этом случае ретинобластома, как правило, сочетается с другими врожденными аномалиями – пороками сердца, расщелинами неба («волчьей пастью»), кортикальным детским гиперостозом и др. У больных с генетической формой ретинобластомы имеется повышенный риск развития злокачественных новообразований другой локализации.

Спорадические случаи ретинобластомы встречаются реже и возникают по неизвестным причинам. К числу факторов риска негенетической формы ретинобластомы принято относить высокий возраст родителей, их занятость на металлургических производствах, неблагоприятные экологические и алиментарные факторы, вызывающие аберрации в хромосомах ретинальных и половых клеток. Данная форма опухоли проявляется у детей более старшего возраста и крайне редко - у взрослых. \n
Лечение ретинобластомы
Лечение ретинобластомы проводится с учетом стадии опухолевого поражения. При этом может использоваться хирургическое вмешательство, лучевая и лекарственная химиотерапия, лазеркоагуляция, криотерапия, термотерапия. Правильное сочетание этих методов во многих случаях позволяет сохранить глаз и зрение. В случает двусторонней ретинобластомы, в отношении каждого глаза определяется дифференцированная тактика в зависимости от степени его поражения опухолью.
                """)
                self.c.resize(600, 700)
                self.c.move(600, 110)
            if max(spisok) == prediction[0][1]:
                self.classification_label.setText('Cataract')
                self.classification_label.move(850, 60)
                self.classification_label.setStyleSheet('color: red;')
                self.d.insertPlainText("""К сожалению, у вас обнаружена катаракта. Катаракта – патология органа  зрения, характеризующаяся частичным или тотальным помутнением хрусталика, который находится между стекловидным телом и радужкой глаза. \n
Причины развития катаракты и группы риска
Заболевание может развиться:

у пожилых людей – возрастная катаракта (90% от всех случаев);
у людей после травм – травматическая катаракта (4%);
после радиационного облучения – лучевая катаракта (3%);
у новорождённых – врождённая катаракта (3%).
Ее развитию способствуют эндокринные расстройства (нарушение обмена веществ, сахарный диабет), авитаминоз, некоторые глазные заболевания, неблагоприятная экологическая обстановка, длительный приём определённых лекарственных препаратов. В последние годы было доказано, что причиной заболевания может также стать активное курение. \n
Лечение катаракты
Сегодня для лечения катаракты врачи применяют комплексный подход, сочетающий медикаментозную терапию и оперативные методы лечения. Используемые препараты способны замедлить или на время приостановить развитие патологического процесса. Однако процесс помутнения хрусталика необратим, на сегодняшний день не существует лекарства, которое могло бы полностью излечить катаракту.

По этой причине единственным действенным методом лечения является ультразвуковая факоэмульсификация катаракты (УЗФЭК) -  хирургическое вмешательство, которое заключается в замене помутневшего хрусталика на искусственную интраокулярную линзу (ИОЛ). УЗФЭК является одной из наиболее распространенных и успешных процедур в  офтальмологии.
Операция занимает  15-20 минут, процедура полностью безболезненна и хорошо переносится пациентами, так как проводится под местной анестезией (с помощью обезболивающих капель). 4 из 5 пациентов сообщили, что операция по удалению катаракты прошла намного легче, чем они ожидали.
                """)
                self.d.resize(600, 700)
                self.d.move(600, 110)
            self.statusBar().showMessage('Изображение классифицировано')
        except AttributeError:
            self.statusBar().showMessage('Сперва выберите изображение')


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ImageClassifier()
    window.show()
    sys.exit(app.exec_())

2023-03-15 23:55:38.039975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 23:55:40.770190: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 23:55:40.771357: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: невозможно открыть разделяемый объектный файл: Нет такого файла или каталога (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: невозможно открыть разделяемый объектный файл: Нет такого файла или каталога (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


1/1 [==============================] - 1s 591ms/step
